### 1. 데이터셋과 데이터 로더 작성

In [30]:
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymeopteraDataset
from torch.utils.data import DataLoader

train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymeopteraDataset(file_list = train_list, transform = ImageTransform(size, mean, std), phase='train')
val_dataset = HymeopteraDataset(file_list = val_list, transform = ImageTransform(size, mean, std),  phase='val')

batch_size = 32
train_dataloder = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True)

dataloader_dict = {'train': train_dataloder, 'val': val_dataloader}

./data/hymenoptera_data\train\**\*.jpg
./data/hymenoptera_data\val\**\*.jpg


### 2. 네트워크 모델 작성

In [31]:
from torchvision.models import vgg16, VGG16_Weights
import torch.nn as nn

net = vgg16(weights = VGG16_Weights.DEFAULT)

net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

net.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 3. 손실함수 정의

In [32]:
criterion = nn.CrossEntropyLoss()

### 4. 최적화 방법 설정

In [33]:
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

update_param_names_1 = ['features']
update_param_names_2 = ['classifier.0.weight', 
                      'classifier.0.bias', 
                      'classifier.3.weight',
                      'classifier.3.bias']
update_param_names_3 = ['classifier.6.weight', 'classifier.6.bias']

for name, param  in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print('params_to_update_1에 저장: ', name)
    
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print('params_to_update_2에 저장: ', name)
    
    elif name in update_param_names_3:
        param.requries_grad = True
        params_to_update_3.append(param)
        print('params_to_update_3에 저장: ', name)
    
    else:
        param.requries_grad = False
        print('경사 계산 없음. 학습하지 않음: ', name)

params_to_update_1에 저장:  features.0.weight
params_to_update_1에 저장:  features.0.bias
params_to_update_1에 저장:  features.2.weight
params_to_update_1에 저장:  features.2.bias
params_to_update_1에 저장:  features.5.weight
params_to_update_1에 저장:  features.5.bias
params_to_update_1에 저장:  features.7.weight
params_to_update_1에 저장:  features.7.bias
params_to_update_1에 저장:  features.10.weight
params_to_update_1에 저장:  features.10.bias
params_to_update_1에 저장:  features.12.weight
params_to_update_1에 저장:  features.12.bias
params_to_update_1에 저장:  features.14.weight
params_to_update_1에 저장:  features.14.bias
params_to_update_1에 저장:  features.17.weight
params_to_update_1에 저장:  features.17.bias
params_to_update_1에 저장:  features.19.weight
params_to_update_1에 저장:  features.19.bias
params_to_update_1에 저장:  features.21.weight
params_to_update_1에 저장:  features.21.bias
params_to_update_1에 저장:  features.24.weight
params_to_update_1에 저장:  features.24.bias
params_to_update_1에 저장:  features.26.weight
params_to_update_1

In [34]:
import torch.optim as optim
import torch

# 최적화 방법 설정
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

### 5. 학습 및 검증 실시

In [35]:
from tqdm import tqdm

def train_model(net, dataloader_dict, criterion, optimizer, num_epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('사용 장치: ', device)
    
    net.to(device)
    
    # 네트워크가 어느 정도 고정되면 고속화시킨다.
    torch.backends.cudnn.benchmark = True
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('---------------')
        
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            # 학습하지 않았을 때의 검증 성능을 확인하기 위해 epoch=0의 훈련 생략
            if (epoch == 0) and (phase == 'train'):
                continue
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    preds = torch.argmax(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += preds.eq(labels).sum().item()
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects / len(dataloader_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))                    

In [36]:
num_epochs = 2
train_model(net, dataloader_dict, criterion, optimizer, num_epochs)

사용 장치:  cuda
Epoch 1/2
---------------


100%|██████████| 5/5 [00:01<00:00,  4.29it/s]


val Loss: 0.7427 Acc: 0.4641
Epoch 2/2
---------------


100%|██████████| 8/8 [00:03<00:00,  2.37it/s]


train Loss: 0.4966 Acc: 0.7325


100%|██████████| 5/5 [00:01<00:00,  4.53it/s]

val Loss: 0.1695 Acc: 0.9608


### 6. 학습된 네트워크 저장 및 로드

In [40]:
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [ ]:
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

load_weights = torch.load(load_path, map_location='cpu')
net.load_state_dict(load_weights)

C:\Users\jyj1206\AppData\Local\Temp\ipykernel_18528\4077220187.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_weights = torch.load(load_path)
C:\Users\jyj1206\AppD

<All keys matched successfully>